# Convolutional Neural Networks

## Project:Landmark Classification


### A simple app that uses the exported model.



The app will show the top 5 classes that the model think are most relevant for the picture you have uploaded

In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import PIL.Image
import torch
import torch.jit
import io
import torchvision.transforms as T

# Load your JIT-exported model
jit_model_path = 'checkpoints/transfer_exported.pt'
jit_model = torch.jit.load(jit_model_path)

# Define the upload button and classification button
upload_button = widgets.FileUpload(accept=".jpg,.jpeg", multiple=True)
classify_button = widgets.Button(description="Classify")
output_image = widgets.Output()
output_scores = widgets.Output()

def classify_button_clicked(b):
    with output_image:
        clear_output(wait=True)
        if upload_button.value:
            for file_upload in upload_button.value:  # Iterate over the list of dictionaries
                img_data = file_upload['content']  # Access the content using the 'content' key
                img = PIL.Image.open(io.BytesIO(img_data))

                # Extract the file name from the dictionary
                file_name = file_upload['name']
                
                # Print the file name
                print(f"File: {file_name}")
                
                # Display a smaller version of the image
                ratio = img.size[0] / img.size[1]
                img_thumb = img.copy()
                img_thumb.thumbnail([ratio * 200, 200])
                display(img_thumb)
    
                img_tensor = preprocess_image(img)
                scores = jit_model(img_tensor).data.cpu()
                _, indices = torch.topk(scores, k=5)
                
                class_names = jit_model.class_names
                
                # Calculate the maximum label length for the top 5 labels
                max_label_length = max(len(class_names[i]) if i < len(class_names) else len(f"Class {i.item()}") for i in indices[0])
                
                print("Top 5 classes and their scores:")
                for i in indices[0]:
                    label = class_names[i] if i < len(class_names) else f"Class {i.item()}"
                    score = scores[0, i].item()
                    padding = max_label_length - len(label) + 1  # Calculate padding based on max label length
                    print(f"{label:<{max_label_length}} Score {score:.2f}")
                print("\n" + "=" * 50 + "\n")
        else:
            print("Please upload an image first.")
                        
def preprocess_image(image):
    # Preprocess the image for the model   
    transform = T.Compose([
        T.Resize(256),
        T.ToTensor()
    ])    
    # Apply the transformation to the image
    img_tensor = transform(image).unsqueeze_(0)
    return img_tensor

classify_button.on_click(classify_button_clicked)

# Display the widgets
display(upload_button)
display(classify_button)
display(output_image)
display(output_scores)

FileUpload(value=(), accept='.jpg,.jpeg', description='Upload', multiple=True)

Button(description='Classify', style=ButtonStyle())

Output()

Output()